In [1]:
# !pip install repeng accelerate datasets

In [2]:
#!/usr/bin/env python3
# ===============================================================
#         RASPID  –  Reasoning-Aware Steering with PID control
#         Robust Implementation with Numerical Stability Fixes
# ===============================================================
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from repeng import ControlModel, ControlVector, DatasetEntry
from datasets import load_dataset
import re
import numpy as np
from tqdm import tqdm
import time

# ---------------- CONFIG ---------------------------------------------------
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float32  # Using full precision for numerical stability
CHUNK_SIZE = 64        # Size of chunks for analysis

# Sampling parameters
TEMP = 0.7
TOP_P = 0.9
REP_PENALTY = 1.2

# Print GPU info
if torch.cuda.is_available():
   for i in range(torch.cuda.device_count()):
       print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
       print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")

# ---------------- LOAD MODEL ----------------------------------------------
print(f"Loading {MODEL_NAME} on {DEVICE} with {DTYPE}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModelForCausalLM.from_pretrained(
   MODEL_NAME, 
   torch_dtype=DTYPE,
   device_map="auto" if DEVICE=="cuda" else None
).eval()

# ---------------- HELPER FUNCTIONS ----------------------------------------
def sanitize(t):
   """Safely clamp tensor values and handle NaNs"""
   return torch.nan_to_num(torch.clamp(t, -100.0, 100.0), nan=0.0)

def safe_entropy(logits):
   """Calculate entropy with extra safeguards against NaN"""
   try:
       # Add small epsilon to prevent log(0)
       p = torch.softmax(logits, dim=-1) + 1e-10
       p = p / p.sum()  # Renormalize
       
       # Stable entropy calculation
       entropy_val = -torch.sum(p * torch.log(p))
       
       # Convert to float and check for valid value
       result = float(entropy_val)
       if np.isnan(result) or np.isinf(result):
           print("Warning: Entropy calculation returned NaN/Inf, using default")
           return 4.0  # Default reasonable entropy
       return result
   except Exception as e:
       print(f"Error in entropy calculation: {e}")
       return 4.0  # Default value

def safe_kl_div(a, b):
   """Calculate KL divergence with safeguards against numerical issues"""
   try:
       # Add small epsilon for numerical stability
       pa = torch.softmax(a, dim=-1) + 1e-10
       pb = torch.softmax(b, dim=-1) + 1e-10
       
       # Renormalize
       pa = pa / pa.sum()
       pb = pb / pb.sum()
       
       # Calculate KL divergence
       kl = torch.sum(pa * (torch.log(pa) - torch.log(pb)))
       
       # Convert to float and check for valid value
       result = float(kl)
       if np.isnan(result) or np.isinf(result):
           print("Warning: KL calculation returned NaN/Inf, using default")
           return 0.01  # Default small KL value
       
       # Clip to reasonable range
       return max(min(result, 10.0), 0.0)
   except Exception as e:
       print(f"Error in KL calculation: {e}")
       return 0.01  # Default value

# ---------------- TRAIN DENSE-REASONING VECTOR ----------------------------
print("\n• Building dense-reasoning control vector...")
control_model = ControlModel(base_model, [20]).to(DEVICE)  # Use layer 20 for control

# Process and prepare training data
ds = load_dataset("rb/aime_reasoning", "default")["train"].select(range(100))
def strip(txt): return re.sub(r"[^\x20-\x7E]", " ", txt)  # ASCII only for training
pairs = [DatasetEntry(strip(r["refined_reasoning"]),
                    strip(r["reasoning_content"])) for r in ds]

# Train the control vector
print("Training control vector with 100 examples...")
ctrl_vec = ControlVector.train(control_model, tokenizer, pairs, batch_size=1)
print("Control vector training complete")

GPU 0: NVIDIA A100-PCIE-40GB
  Memory: 42.41 GB
Loading deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B on cuda with torch.float32...


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.



• Building dense-reasoning control vector...
Training control vector with 100 examples...


100%|██████████| 27/27 [00:00<00:00, 53.46it/s]

Control vector training complete


In [3]:
MAX_TOKENS = 2048       # Maximum generation length

# ---------------- BASELINE GENERATION FUNCTION ---------------------------
def generate_baseline(prompt, max_tokens=MAX_TOKENS):
   """Reliable baseline generation using HuggingFace's generate method"""
   print(f"\nRunning baseline generation...")
   
   # Encode prompt and setup
   inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
   input_len = len(inputs.input_ids[0])
   
   # Generate with standard parameters
   print("Starting generation...")
   outputs = base_model.generate(
       inputs.input_ids,
       max_new_tokens=max_tokens,
       do_sample=True,
       temperature=TEMP,
       top_p=TOP_P,
       repetition_penalty=REP_PENALTY,
       pad_token_id=tokenizer.eos_token_id,
       attention_mask=inputs.attention_mask
   )
   
   # Decode and return
   generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
   new_tokens = len(outputs[0]) - input_len
   
   print(f"Baseline generation complete: {new_tokens} new tokens generated")
   return generated_text, new_tokens

In [8]:
@torch.inference_mode()
def generate_raspid_improved(prompt, max_tokens=MAX_TOKENS):
    """Improved RASPID generation with adaptive control and safety mechanisms"""
    print(f"\nRunning improved RASPID generation...")

    # More conservative parameters - lower influence overall
    KP_base = 0.3  # Lower P-term to prevent overshoot
    KI_base = 0.03  # Slightly higher I-term to maintain state
    KD_base = 0.05  # Higher D-term to dampen oscillations
    
    # More appropriate setpoints based on reasoning patterns
    KL_setpoint = 0.1  # Lower KL target - less divergence for coherence
    ENT_setpoint = 2.0  # Higher entropy target - more exploration
    
    print(f"Base parameters: KP={KP_base:.2f}, KI={KI_base:.2f}, KD={KD_base:.2f}")
    print(f"Setpoints: KL={KL_setpoint:.2f}, ENT={ENT_setpoint:.2f}")

    # Encode prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to(DEVICE).input_ids[0]
    prompt_len = len(input_ids)

    # PID controller state
    integ = 0.0
    deriv = 0.0
    prev_err = 0.0
    alpha = 0.0
    max_alpha_history = 0.0  # Track historical maximum alpha
    
    # Performance tracking
    repeated_ngrams = set()
    recent_tokens = []
    off_track_count = 0

    # Chunk tracking
    chunk_sum = None
    tok_in_chunk = 0
    last_vec = None
    
    # Vector history for quality monitoring
    vector_history = []
    content_quality = 1.0  # Quality score (1.0 = good, 0.0 = bad)

    # Output tracking
    output_ids = input_ids.clone()
    generated = []
    past = None

    # For logging PID controller performance
    pid_log = []
    
    # Generation parameters
    initial_tokens = 100  # Longer unguided phase for context buildup
    transition_window = 50  # Gradual transition window
    steering_active = False
    steering_intensity = 0.0  # Gradual ramp-up

    # Generation loop
    pbar = tqdm(total=max_tokens, desc="Improved RASPID")
    try:
        for i in range(max_tokens):
            # Determine steering state
            tokens_generated = len(generated)
            
            # Gradual transition to steering
            if tokens_generated >= initial_tokens and tokens_generated < (initial_tokens + transition_window):
                if not steering_active:
                    print(f"\n----- BEGINNING GRADUAL TRANSITION TO STEERING -----")
                    steering_active = True
                
                # Linear ramp-up of steering intensity
                steering_intensity = min(1.0, (tokens_generated - initial_tokens) / transition_window)
            elif tokens_generated >= (initial_tokens + transition_window):
                steering_intensity = 1.0
            
            # Apply control only if steering is active with meaningful alpha
            if steering_active and alpha > 0.01:
                # Apply smooth, sigmoid-scaled alpha for more gradual influence
                sigmoid_alpha = 1.0 / (1.0 + math.exp(-6 * (alpha - 0.5)))  # Sigmoid centered at 0.5
                effective_alpha = min(0.3, sigmoid_alpha * steering_intensity * alpha)
                
                if i % 50 == 0 or effective_alpha > 0.15:
                    print(f"Step {i}: Alpha={alpha:.3f}, Effective={effective_alpha:.3f}, Quality={content_quality:.2f}")
                
                control_model.set_control(ctrl_vec, coeff=effective_alpha)
            else:
                control_model.reset()
            
            # Forward pass through model
            with torch.no_grad():
                outputs = control_model(
                    input_ids=output_ids[-1:].unsqueeze(0),
                    past_key_values=past,
                    use_cache=True
                )
                past = outputs.past_key_values
                
                # Get logits with safety measures
                logits = sanitize(outputs.logits[0, -1, :])
                
                # IMPROVEMENT: Detect repetition patterns using n-grams
                if tokens_generated > 10:
                    # Detect 4-gram repetition (can adjust n-gram size)
                    recent_tokens = generated[-20:]  # Look at last 20 tokens
                    
                    # Create 4-grams
                    for j in range(len(recent_tokens) - 4):
                        ngram = tuple(recent_tokens[j:j+4])
                        if ngram in repeated_ngrams:
                            # Detected repetition, penalize
                            off_track_count += 1
                            content_quality *= 0.95  # Degrade quality score
                            if off_track_count > 5:
                                print(f"WARNING: Repetition detected, reducing control strength")
                                alpha *= 0.7  # Reduce steering when repetitive
                        else:
                            repeated_ngrams.add(ngram)
                
                # Accumulate for chunk analysis
                chunk_sum = logits.clone() if chunk_sum is None else chunk_sum + logits
                tok_in_chunk += 1
                
                # Process completed chunk
                if tok_in_chunk >= CHUNK_SIZE:
                    # Calculate chunk vector and entropy
                    vec = chunk_sum / tok_in_chunk
                    H = safe_entropy(logits)
                    
                    # Store vector for quality monitoring
                    if len(vector_history) < 5:
                        vector_history.append(vec)
                    else:
                        vector_history.pop(0)
                        vector_history.append(vec)
                    
                    # IMPROVEMENT: Monitor content quality through vector consistency
                    if len(vector_history) >= 3:
                        # Check if vectors are becoming too similar (stuck in a pattern)
                        # or too different (going off track)
                        kl_prev_2 = safe_kl_div(vector_history[-1], vector_history[-3])
                        if kl_prev_2 < 0.01 or kl_prev_2 > 1.0:
                            off_track_count += 1
                            content_quality *= 0.9
                            print(f"Warning: Content quality degrading: {content_quality:.2f}, KL={kl_prev_2:.3f}")
                        else:
                            # Gradually recover quality when generation is good
                            content_quality = min(1.0, content_quality * 1.05)
                            off_track_count = max(0, off_track_count - 1)
                    
                    if last_vec is not None:
                        # Calculate KL divergence with safety
                        kl = safe_kl_div(vec, last_vec)
                        
                        # IMPROVEMENT: Reset or reduce steering if KL is too high (going off track)
                        if kl > 0.8 and steering_active and tokens_generated > initial_tokens + 100:
                            print(f"WARNING: High KL={kl:.3f} detected, reducing steering")
                            alpha *= 0.7
                            integ *= 0.5  # Reset integrator partially
                        
                        # Only apply PID control if steering is active
                        if steering_active:
                            # Adaptive weighting based on progress & quality
                            progress = min(1.0, tokens_generated / 300.0)
                            kl_weight = 0.3 + 0.2 * progress
                            ent_weight = 0.7 - 0.3 * progress
                            
                            # IMPROVEMENT: Scale weights by content quality
                            kl_weight *= content_quality
                            ent_weight *= content_quality
                            
                            # Compute weighted error - prioritize entropy early, KL later
                            err = kl_weight * (kl - KL_setpoint) + ent_weight * (ENT_setpoint - H)
                            
                            # IMPROVEMENT: Adaptive PID parameters based on generation quality
                            # When quality is low, reduce all parameters to back off control
                            KP = KP_base * content_quality
                            KI = KI_base * content_quality
                            KD = KD_base * content_quality
                            
                            # Update integral term with anti-windup and leaky integration
                            # Apply proportional leakage to prevent integral windup
                            integ = 0.90 * integ + err
                            
                            # Improved clamping with variable bounds - more restricted when quality is low
                            max_i = 0.5 * content_quality
                            integ = max(min(integ, max_i), -max_i)
                            
                            # Calculate derivative with better smoothing
                            deriv = 0.7 * deriv + 0.3 * (err - prev_err)
                            prev_err = err
                            
                            # Calculate alpha with adaptive parameters and dynamic damping
                            # Lower damping when quality is good, more damping when problematic
                            damping = 0.5 + 0.5 * content_quality
                            
                            # IMPROVEMENT: More balanced term contribution
                            p_term = KP * err
                            i_term = KI * integ
                            d_term = KD * deriv
                            
                            raw_alpha = damping * (p_term + i_term + d_term)
                            
                            # Progressive, quality-aware clamping for alpha
                            # Lower max limit when quality is poor
                            max_alpha = 0.3 * content_quality + 0.1 * progress
                            alpha = max(min(raw_alpha, max_alpha), 0.0)
                            
                            # Track the historical maximum alpha for detecting issues
                            if alpha > max_alpha_history:
                                max_alpha_history = alpha
                            
                            # IMPROVEMENT: Detect alpha saturation (sign of control issues)
                            if alpha >= max_alpha and alpha > 0.2:
                                # Alpha is hitting the ceiling too often
                                print(f"NOTE: Alpha saturation detected: {alpha:.3f}")
                                
                                # Back off control to prevent runaway steering
                                if content_quality < 0.8:
                                    print(f"WARNING: Quality issues with high alpha, reducing by 20%")
                                    alpha *= 0.8
                                    integ *= 0.7  # Reset integrator partially
                            
                            # Log PID state
                            pid_log.append({
                                'step': i,
                                'tokens_generated': tokens_generated,
                                'kl': kl,
                                'entropy': H,
                                'error': err,
                                'content_quality': content_quality,
                                'progress': progress,
                                'kl_weight': kl_weight,
                                'ent_weight': ent_weight,
                                'p_term': p_term,
                                'i_term': i_term,
                                'd_term': d_term,
                                'alpha': alpha,
                                'effective_alpha': effective_alpha if steering_active and alpha > 0.01 else 0
                            })
                    
                    # Reset for next chunk
                    last_vec = vec
                    chunk_sum = None
                    tok_in_chunk = 0
                
                # IMPROVEMENT: Adaptive temperature based on quality and steering
                # - Higher temperature when quality is low (exploration)
                # - Lower temperature when steering actively (precision)
                temperature = TEMP * (1.1 if content_quality < 0.7 else 
                                    (0.95 if steering_active and alpha > 0.1 else 1.0))
                
                # IMPROVEMENT: Incorporate repetition penalty dynamically
                rep_penalty = REP_PENALTY
                if off_track_count > 3:
                    rep_penalty = REP_PENALTY * (1.0 + 0.1 * off_track_count)
                    if i % 50 == 0:
                        print(f"Increased repetition penalty to {rep_penalty:.2f}")
                
                # Apply penalties to logits directly for problematic tokens if needed
                if off_track_count > 5:
                    # Identify tokens that are overused in recent output and penalize
                    token_counts = {}
                    for tok in recent_tokens:
                        token_counts[tok] = token_counts.get(tok, 0) + 1
                    
                    # Penalize overused tokens
                    for tok, count in token_counts.items():
                        if count > 3:  # Token used more than 3 times recently
                            logits[tok] -= 1.0 * count  # Progressive penalty
                
                # Sample token with adjusted parameters
                probs = torch.softmax(logits / temperature, dim=-1)
                next_token = torch.multinomial(probs, 1).item()
                
                # Add to output
                output_ids = torch.cat([output_ids, torch.tensor([next_token], device=DEVICE)])
                generated.append(next_token)
                pbar.update(1)
                
                # IMPROVEMENT: Emergency intervention if going off-track for too long
                if off_track_count > 10:
                    print(f"EMERGENCY: Generation appears to be off-track, resetting control")
                    control_model.reset()
                    alpha = 0.0
                    integ = 0.0
                    deriv = 0.0
                    off_track_count = 0
                    content_quality = 0.6  # Set to medium-low but allow recovery
                
                # Stop on EOS token
                if next_token == tokenizer.eos_token_id:
                    print("EOS token detected, stopping generation")
                    break
    except Exception as e:
        import traceback
        print(f"Error during generation: {e}")
        traceback.print_exc()
    finally:
        pbar.close()
        control_model.reset()

    # Decode output
    generated_text = tokenizer.decode(output_ids, skip_special_tokens=True)
    new_tokens = len(output_ids) - prompt_len

    print(f"Improved RASPID generation complete: {new_tokens} new tokens generated")

    # Print PID controller analysis
    if pid_log:
        print("\n----- PID CONTROLLER ANALYSIS -----")
        # Component strength analysis
        p_values = [abs(entry['p_term']) for entry in pid_log]
        i_values = [abs(entry['i_term']) for entry in pid_log] 
        d_values = [abs(entry['d_term']) for entry in pid_log]
        
        print(f"P-term avg magnitude: {sum(p_values)/len(p_values):.4f}")
        print(f"I-term avg magnitude: {sum(i_values)/len(i_values):.4f}")
        print(f"D-term avg magnitude: {sum(d_values)/len(d_values):.4f}")
        
        # Component ratios
        p_sum = sum(p_values)
        i_sum = sum(i_values)
        d_sum = sum(d_values)
        total = p_sum + i_sum + d_sum
        
        print(f"Component contribution: P={p_sum/total*100:.1f}%, I={i_sum/total*100:.1f}%, D={d_sum/total*100:.1f}%")
        
        # Standard stats
        kl_values = [entry['kl'] for entry in pid_log]
        alpha_values = [entry['alpha'] for entry in pid_log]
        effective_alphas = [entry['effective_alpha'] for entry in pid_log]
        entropy_values = [entry['entropy'] for entry in pid_log]
        quality_values = [entry['content_quality'] for entry in pid_log]
        
        print(f"KL divergence: min={min(kl_values):.4f}, avg={sum(kl_values)/len(kl_values):.4f}, max={max(kl_values):.4f}")
        print(f"Entropy: min={min(entropy_values):.2f}, avg={sum(entropy_values)/len(entropy_values):.2f}, max={max(entropy_values):.2f}")
        print(f"Alpha values: min={min(alpha_values):.4f}, avg={sum(alpha_values)/len(alpha_values):.4f}, max={max(alpha_values):.4f}")
        print(f"Effective alphas: min={min(effective_alphas):.4f}, avg={sum(effective_alphas)/len(effective_alphas):.4f}, max={max(effective_alphas):.4f}")
        print(f"Content quality: min={min(quality_values):.2f}, avg={sum(quality_values)/len(quality_values):.2f}, max={max(quality_values):.2f}")

    return generated_text, new_tokens


In [ ]:
# ---------------- RUN BENCHMARK ------------------------------------------
print("\n• Running benchmark with duck eggs problem...")
test_problem = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
prompt = f"{test_problem}\n\nAnswer step by step and end with: Final answer: \\boxed{{numeric_value}}"
# Run both generators
baseline_text, baseline_tokens = generate_baseline(prompt)
raspid_text, raspid_tokens = generate_raspid(prompt)

# Calculate token savings
token_saving = (1 - raspid_tokens/baseline_tokens)*100

# Display results
print("\n──────── SUMMARY ───────────────")
print(f"BASE: {baseline_tokens} new tokens")
print(f"RAST: {raspid_tokens} new tokens") 
print(f"Token saving: {token_saving:.1f}%")
print("─────────────────────────────────")

print("\n──────── BASELINE OUTPUT ───────────────")
print(baseline_text)
print("\n──────── RASPID OUTPUT ───────────────")
print(raspid_text)
print("─────────────────────────────────")


• Running benchmark with duck eggs problem...

Running baseline generation...
Starting generation...
Baseline generation complete: 723 new tokens generated

Running RASPID generation with analytically tuned parameters...
Analytically tuned parameters: KP=0.80, KI=0.01, KD=0.01
Adjusted setpoints: KL=0.2000, ENT=1.00


RASPID generation:   0%|          | 5/2048 [00:00<00:44, 45.50it/s]

Logit stats: min=-12.12, max=14.04


RASPID generation:   3%|▎         | 60/2048 [00:01<00:44, 44.94it/s]

Logit stats: min=-10.36, max=28.71


...

Step 935: Alpha=0.3742, Effective=0.1589
Step 936: Alpha=0.3742, Effective=0.1589
Step 937: Alpha=0.3742, Effective=0.1589
Step 938: Alpha=0.3742, Effective=0.1589
Step 939: Alpha=0.3742, Effective=0.1589
Step 940: Alpha=0.3742, Effective=0.1589
Step 941: Alpha=0.3742, Effective=0.1589
Step 942: Alpha=0.3742, Effective=0.1589
Step 943: Alpha=0.3742, Effective=0.1589


RASPID generation:  46%|████▋     | 950/2048 [00:21<00:24, 44.37it/s]

Step 944: Alpha=0.3742, Effective=0.1589
Step 945: Alpha=0.3742, Effective=0.1589
Step 946: Alpha=0.3742, Effective=0.1589
Step 947: Alpha=0.3742, Effective=0.1589
Step 948: Alpha=0.3742, Effective=0.1589
Step 949: Alpha=0.3742, Effective=0.1589
Step 950: Alpha=0.3742, Effective=0.1589
Logit stats: min=-7.09, max=24.97
Step 951: Alpha=0.3742, Effective=0.1589
Step 952: Alpha=0.3742, Effective=0.1589


RASPID generation:  47%|████▋     | 960/2048 [00:21<00:24, 44.30it/s]

Step 953: Alpha=0.3742, Effective=0.1589
Step 954: Alpha=0.3742, Effective=0.1589
Step 955: Alpha=0.3742, Effective=0.1589
Step 956: Alpha=0.3742, Effective=0.1589
Step 957: Alpha=0.3742, Effective=0.1589
Step 958: Alpha=0.3742, Effective=0.1589
Step 959: Alpha=0.3742, Effective=0.1589
Step 960: Alpha=0.1302, Effective=0.0612
Step 961: Alpha=0.1302, Effective=0.0612
